In [ ]:
!pip3 install grad-cam

In [ ]:
import cv2
import numpy as np
import torch
from PIL import Image
import pydicom
import pandas as pd
from torchvision import models
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM, LayerCAM
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import deprocess_image, \
    preprocess_image


In [ ]:
def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    x = heatmap
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam), x

In [ ]:
root_path = 'C:/Users/tcemcetin/Documents/YüksekLisans/TrustableAIProject/'
df = pd.read_csv(root_path + 'dataset/stage_2_train_labels.csv')
df = df[df.Target == 1]
df['area'] = df.apply(lambda r: r.height * r.width, axis=1)
df = df.sort_values(['patientId', 'area'], ascending=False).drop_duplicates('patientId', keep='first')
df

In [ ]:
def get_model(model_name):
    """
      Choose the target layer you want to compute the visualization for.
      Usually this will be the last convolutional layer in the model.
      Some common choices can be:
      Resnet18 and 50: model.layer4[-1]
      VGG, densenet161: model.features[-1]
      mnasnet1_0: model.layers[-1]
      You can print the model to help chose the layer
      You can pass a list with several target layers,
      in that case the CAMs will be computed per layer and then aggregated.
      You can also try selecting all layers of a certain type, with e.g:
      from pytorch_grad_cam.utils.find_layers import find_layer_types_recursive
      find_layer_types_recursive(model, [torch.nn.ReLU])
    """

    
    if model_name == "resnet_50_raw":
        model = models.resnet50(pretrained=True)
        target_layers = [model.layer4[-1]]
    
    elif model_name == "densenet161_raw":
        model = models.densenet161(pretrained=True)
        target_layers = [model.features[-1]]

    elif model_name == "resnet_special":
        model = torch.load('./models/resnet_fold1/checkpoint_38.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.layer4[-1]]
        
    elif model_name == "densenet_special":
        model = torch.load('./models/densenet_fold1/checkpoint_31.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.features[-1]]
    
    elif model_name == "resnet_special_v2":
        model = torch.load('./models/resnet_fold0/checkpoint_10.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.layer4[-1]]
        
    elif model_name == "densenet_special_v2":
        model = torch.load('./models/densenet_fold0/checkpoint_10.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.features[-1]]
        
    elif model_name == "densenet_special_v3":
        model = torch.load('./models/densenet_fold3/checkpoint_6.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.features[-1]]
    
    elif model_name == "resnet_special_v3":
        model = torch.load('./models/densenet_fold2/checkpoint_7.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.layer4[-1]]
        
    elif model_name == "densenet_model_fold0":
        model = torch.load(f'./models/models/DenseNet/dense_fold0/checkpoint_10.pth.tar', map_location=torch.device('cpu'))['model']
        target_layers = [model.features[-1]]
    else:
        return None, None

    return model, target_layers

In [ ]:
def load_img(image_path):
    dcm_file = pydicom.read_file(image_path)
    img_arr = dcm_file.pixel_array
    img = Image.fromarray(img_arr).convert('RGB')
    return img


def preprocess(img):
    img_raw = img.copy()
    if img_raw.size[0] > img_raw.size[1]:
        img_raw.thumbnail((1000000, 256))
    else:
        img_raw.thumbnail((256 ,1000000))
    
    Left = (img_raw.width - 224) / 2
    Right = Left + 224
    Top = (img_raw.height - 244) / 2
    Buttom = Top + 224
    img_raw = img_raw.crop((Left, Top, Right, Buttom))
    
    return img_raw

def create_mask(row):
    mask = np.zeros((1024,1024), dtype=int)
    y = int(row['y'])
    y_max = int(row['y']+row['height']+1)
    x =  int(row['x'])
    x_max = int(row['x']+row['width']+1)
    mask[y:y_max+1, x:x_max] = 1
    return mask, (y, x), (y_max, x_max)
        

def calculate_IOU(mask, heatmap):
    overlap = mask*heatmap # Logical AND
    union = mask + heatmap # Logical OR
    IOU = overlap.sum()/float(union.sum())
    return IOU

def calculate_dice_score(mask, heatmap, k=1):
    return np.sum(heatmap[mask==k])*2.0 / (np.sum(heatmap) + np.sum(mask))

    
def gcam(config, model, target_layers):
    methods = \
        {"gradcam": GradCAM,
          "scorecam": ScoreCAM,
          "gradcam++": GradCAMPlusPlus,
          "ablationcam": AblationCAM,
          "xgradcam": XGradCAM,
          "eigencam": EigenCAM,
          "eigengradcam": EigenGradCAM,
          "layercam": LayerCAM,
          "fullgrad": FullGrad}

    # rgb_img = cv2.imread(args.image_path, 1)[:, :, ::-1]
    # rgb_img = np.float32(rgb_img) / 255
    # input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = load_img(config['root_path'] + config['img'] + ".dcm")
    rgb_img = preprocess(img)
    rgb_img = np.float32(rgb_img) / 255
    
    
    input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    target_category = None

    # Using the with statement ensures the context is freed, and you can
    # recreate different CAM objects in a loop.
    cam_algorithm = methods[config['method']]
    with cam_algorithm(model=model,
                        target_layers=target_layers,
                        use_cuda=config['use_cuda']) as cam:

        # AblationCAM and ScoreCAM have batched implementations.
        # You can override the internal batch size for faster computation.
        cam.batch_size = 32

        grayscale_cam = cam(input_tensor=input_tensor,
                            target_category=target_category,
                            aug_smooth=config['aug_smooth'],
                            eigen_smooth=config['eigen_smooth'])
        
        # Here grayscale_cam has only one image in the batch
        grayscale_cam = grayscale_cam[0, :]
        
        cam_image,heatmap = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
        # cam_image is RGB encoded whereas "cv2.imwrite" requires BGR encoding.
        cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)

    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=config['use_cuda'])
    gb = gb_model(input_tensor, target_category=target_category)

    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image(cam_mask * gb)
    gb = deprocess_image(gb)

    return cam_image, gb, cam_gb, heatmap, img

In [ ]:
config = {
    "root_path": "C:/Users/tcemcetin/Documents/YüksekLisans/TrustableAIProject/dataset/stage_2_train_images/",
    "img": "00b4ac1b-fa09-4dbe-b93f-7d9e52992a68",
    "aug_smooth": True,
    "eigen_smooth": False,
    "method": "xgradcam",
    "use_cuda": False
}

model_name = "densenet_special"
model, target_layers = get_model(model_name)
print("Model Loaded")

cam_image, gb, cam_gb, heatmap, img = gcam(config, model, target_layers)
# plt.imshow(cam_image)

In [ ]:
models = ["densenet_special_v2", "resnet_special_v2"]
config = {
    "root_path": "C:/Users/tcemcetin/Documents/YüksekLisans/TrustableAIProject/dataset/stage_2_train_images/",
    "img": "00b4ac1b-fa09-4dbe-b93f-7d9e52992a68",
    "aug_smooth": True,
    "eigen_smooth": False,
    "method": "gradcam",
    "use_cuda": False,
    "th": 150
}
liste = []
cnt = 0
for model_name in models[:1]:
    model, target_layers = get_model(model_name)
    print("Model Loaded")
    
    for index, row in df.iterrows():
        print(row['patientId'])
        config['img'] = row['patientId']
        mask, start_point, end_point = create_mask(row)
        cam_image, gb, cam_gb, heatmap, img = gcam(config, model, target_layers)
        
        # Heatmap oversize
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
        heatmap = cv2.resize(heatmap, (1024,1024), interpolation=cv2.INTER_AREA)
        # heatmap = (heatmap > 180).astype(int)
        
        # Draw rectangle
        rectangle = cv2.rectangle(cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR), start_point, end_point, (255, 0, 0), 2)
        plt.imshow(rectangle)
        plt.show()
        
        IOU = calculate_IOU(mask, heatmap)
        liste.append(IOU)
        plt.imshow(cam_image)
        plt.show()
        cnt += 1
        if cnt > 200:
            break

In [ ]:
models = ["densenet_special_v2", "resnet_special_v2"]
config = {
    "root_path": "C:/Users/tcemcetin/Documents/YüksekLisans/TrustableAIProject/dataset/rsna-pneumonia-detection-challenge/stage_2_train_images/",
    "img": "00b4ac1b-fa09-4dbe-b93f-7d9e52992a68",
    "aug_smooth": True,
    "eigen_smooth": False,
    "method": "layercam",
    "use_cuda": False,
    "th": 150
}
liste2 = []
cnt = 0
for model_name in models[:1]:
    model, target_layers = get_model(model_name)
    print("Model Loaded")

    for index, row in df.iterrows():
        print(row['patientId'])
        config['img'] = row['patientId']
        mask, start_point, end_point = create_mask(row)
        cam_image, gb, cam_gb, heatmap, img = gcam(config, model, target_layers)

        # Heatmap oversize
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
        heatmap = cv2.resize(heatmap, (1024,1024), interpolation=cv2.INTER_AREA)
        heatmap_th = (heatmap > 180).astype(int)

        # Draw rectangle
        rectangle = cv2.rectangle(cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR), start_point, end_point, (255, 0, 0), 2)
        plt.imshow(rectangle)
        plt.show()
        
        plt.imshow(heatmap)
        plt.show()
        
        plt.imshow(heatmap_th)
        plt.show()

        IOU = calculate_IOU(mask, heatmap)
        IOU = calculate_IOU(mask, heatmap_th)
        liste2.append(IOU)
        plt.imshow(cam_image)
        plt.show()
        cnt += 1
        if cnt > 200:
            break

In [ ]:
# mask the image and create the new histogram
histogram, bin_edges = np.histogram(heatmap, bins=100, range=(0.0, 255))

# configure and draw the histogram figure
plt.figure()

plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixel count")
plt.xlim([0.0, 255])
plt.plot(bin_edges[25:-1], histogram[25:])

plt.show()

In [ ]:
from statistics import mean
mean(liste)

In [ ]:
from statistics import mean
mean(liste2)

### Inference

In [ ]:
from tqdm import tqdm
import numpy as np
from torchvision import transforms,models
import torch
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
import os
from train import train_function, save_checkpoint
from test import test_function
from pneumonia import Pneumonia
import pandas as pd
from train import calculateMetrics

In [ ]:
class_to_idx = {'Normal': 0, 'Lung Opacity': 1}
cat_to_name = {class_to_idx[i]: i for i in list(class_to_idx.keys())}

In [ ]:
data_transforms = {
            'train': transforms.Compose([
                        transforms.Resize((224, 224)),
                        transforms.CenterCrop(224),
                        transforms.RandomHorizontalFlip(), # randomly flip and rotate
                        transforms.RandomRotation(10),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                    ]),
    
            'test': transforms.Compose([
                        transforms.Resize((224,224)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                        ]),
    
            'valid': transforms.Compose([
                        transforms.Resize((224,224)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                    ])
            }



In [ ]:
device = torch.device('cpu') 
batch_size = 1
train_on_gpu = True
model, _ = get_model('densenet_special_v3')


valid_data = Pneumonia('X_test_fold_0.txt', class_to_idx=class_to_idx, transforms=data_transforms['train'])
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=0, shuffle=True)

model.eval()
number_correct, number_data = 0, 0
for data, target in tqdm(valid_loader):
    if train_on_gpu:
        data, target = data.to(device), target.to(device)
    output = torch.squeeze(model(data))
    pred = output
    print(pred)
    break


### Lime

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
valid_data = Pneumonia('X_test_fold_0.txt', class_to_idx=class_to_idx)
img, label = valid_data.__getitem__(5)
model, _ = get_model('densenet_special_v3')

In [ ]:
img

In [ ]:
def get_pil_transform(): 
    transf = transforms.Compose([
        transforms.Resize((224, 224))
    ])    

    return transf

def get_preprocess_transform():
    transf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])    

    return transf    

pill_transf = get_pil_transform()
preprocess_transform = get_preprocess_transform()

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch = batch.to(device)
    
    logits = model(batch)
    result = []
    
    for l in logits.detach().cpu().numpy():
        if l >= 0.5:
            result.append((1-l, l))
        else:
            result.append((l, 1-l))
    result = np.array(result)
    # probs = F.softmax(logits, dim=1)
    #return probs.detach().cpu().numpy()
    return np.squeeze(result, axis=2)

In [ ]:
test_pred = batch_predict([pill_transf(img)])
test_pred.squeeze().argmax()

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(pill_transf(img)), 
                                         batch_predict, # classification function
                                         top_labels=2, 
                                         hide_color=0, 
                                         num_samples=400)

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
img_boundry1 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry1)

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
img_boundry2 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry2)

In [ ]:
for i in range(100):
    tmp, _ = np.histogram(img, bins=50, range=(0.0, 255))
    histogram += tmp

In [ ]:
df.sample(n=3, random_state=42)

In [ ]:
histogram, bin_edges = np.histogram(img, bins=20, range=(0.0, 255))

plt.figure()

plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixel count")
plt.xlim([0.0, 255])
plt.plot(bin_edges[0:-1], histogram)
plt.savefig('foo.png')

###  -------------------------